In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_core.tools import BaseTool
from langchain import hub

In [8]:
"""You are a D&D dungeon master and a powerful storyteller. You are a student of Joseph Campbell and you know how to structure the beats of an adventure.

You will design a story involving 2 to 4 thematic story hooks. In keeping with the teachings of Joseph Campbell, the drama should increase as the story continues.
The setting and party for the adventure is: {input}

You have the following tools:
StoryFormatter - use this to structure the story in the desired output format

Output Format: return an array of the story hooks in the adventure. Use `StoryFormatter` to structure them into the desired format.
{agent_scratchpad}
"""


'You are a D&D dungeon master and a powerful storyteller. You are a student of Joseph Campbell and you know how to structure the beats of an adventure.\n\nYou will design a story involving 2 to 4 thematic story hooks. In keeping with the teachings of Joseph Campbell, the drama should increase as the story continues.\nThe setting and party for the adventure is: {input}\n\nYou have the following tools:\nStoryFormatter - use this to structure the story in the desired output format\n\nOutput Format: return an array of the story hooks in the adventure. Use `StoryFormatter` to structure them into the desired format.\n{agent_scratchpad}\n'

In [43]:
system_prompt_template = """
You are an experienced dungeon-master for Dungeons & Dragons 5th edition. You will design an adventure involving 2 to 4 thematic story hooks.
The drama and difficulty should increase as the story continues. Each encounter should build off the previous one, resulting in a rousing climax for the players.
The setting and party for the adventure is: {input}

Each encounter should have a difficulty level that is appropriate for the party's strength and the desired dramatic effect of the story hook. For each encounter,
you should select 1 or more enemies such that:
- the sum of the XP of all enemies falls within the range of the desired difficulty level
- the creatures are an evocative part of the narrative and setting the user has chosen
Use the `DifficultyMap` tool to determine the allowable range of enemy XP for the players and their respective levels.
IMPORTANT: The total XP for each encounter MUST NOT exceed the maximum of this range!

Output: Provide a list of creatures or adversaries, including their CRs and XP. 
"""

In [44]:
"""*Procedure:*

*Step One: Initiative*

1. *Initiative Roll:* Use the `TalkTo*` functions to ask each non-player participant for their Initiative, Health points, status effects, etc.
For the player, use `TalkToPlayerAssistant`, incorporating the player's character name to retrieve their Initiative and other relevant stats.
2. *Collecting Initiative Results:* Receive and record the Initiative from each participant, arranging them in descending order, from highest to lowest.

*Step Two: Combat*

1. *Action (Adhering to Initiative Order):*  Starting with the participant at the top of the initiative order, engage with each using the appropriate method. For non-player
characters, use the `TalkTo*` functions. For the player character, use `TalkToPlayerAssistant` and address them by their character name, asking for their action.
2. *Resolving Action (Complete Actions):* When a participant declares their action, follow the appropriate procedure:
    - For non-players, prompt them to roll for attack rolls, damage rolls, skill checks, etc., using `TalkTo*`.
    - For the player, use `TalkToPlayerAssistant` to interact with their character sheet, prompt for rolls and actions, and ensure actions like MultiAttack are fully resolved.
    - Example: If the participant wishes to make an attack, prompt them to roll an attack roll.
    - Example: If a creature uses MultiAttack, prompt for each attack separately and resolve them before proceeding.
3. *Determine Outcome:* Adjudicate the results and describe the outcome of the action. Using the TalkTo* functions, prompt the target for any attributes relevant
to the outcome such as Armor Class, saving throws, etc. Also, prompt participants for any additional rolls if necessary.
    - Example: If a participant has been attacked, ask them for their AC and compare it to the attack roll to determine if the attack was successful.
    - Example: If a participant has succeeded in their attack roll, prompt them to roll for damage.
4. *Notify Affected Participants:* Inform any affected participants of any damage or status effects.

*Step Three: State Tracking*

1. *Update State:* Output the conditions of all participants, including health, conditions (prone, unconscious, and so on).
For the player, use `TalkToPlayerAssistant` to accurately reflect their character's current state.

*Repeat Steps Two and Three until the encounter has ended.*"""

"*Procedure:*\n\n*Step One: Initiative*\n\n1. *Initiative Roll:* Use the `TalkTo*` functions to ask each non-player participant for their Initiative, Health points, status effects, etc.\nFor the player, use `TalkToPlayerAssistant`, incorporating the player's character name to retrieve their Initiative and other relevant stats.\n2. *Collecting Initiative Results:* Receive and record the Initiative from each participant, arranging them in descending order, from highest to lowest.\n\n*Step Two: Combat*\n\n1. *Action (Adhering to Initiative Order):*  Starting with the participant at the top of the initiative order, engage with each using the appropriate method. For non-player\ncharacters, use the `TalkTo*` functions. For the player character, use `TalkToPlayerAssistant` and address them by their character name, asking for their action.\n2. *Resolving Action (Complete Actions):* When a participant declares their action, follow the appropriate procedure:\n    - For non-players, prompt them t

In [45]:
from pydantic import BaseModel, Field
from typing import List, Type, Optional
import functions.encounter_checker as ec

class DifficultyArgs(BaseModel):
    player_levels: List[int] = Field("an integer list of the players in the party")
    desired_difficulty: str = Field("the desired difficulty")

class DifficultyMap(BaseTool):
    name = "DifficultyMap"
    description = "returns total XP for an encounter for a party of adventurers"
    args_schema: Type[BaseModel] = DifficultyArgs

    def _run(
        self, player_levels: List[int], desired_difficulty: str
    ):
        """Use the tool."""
        return ec.calculate_party_xp(player_levels, desired_difficulty)
    
    async def _arun(
        self,
        player_levels: List[int],
        desired_difficulty: str
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("does not support async")

In [49]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
tools = [
    DifficultyMap()
]

prompt.messages[0].prompt.template = system_prompt_template
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [50]:
res = agent_executor.invoke(
    {
        "input": "Party: a level 1 fighter and a level 1 ranger. Setting: a busy marketplace",
        "chat_history": [
            # here is my character sheet!
        ],
    }
)



> Entering new AgentExecutor chain...

Invoking: `DifficultyMap` with `{'player_levels': [1, 1], 'desired_difficulty': 'easy'}`


(0, 50)
Invoking: `DifficultyMap` with `{'player_levels': [1, 1], 'desired_difficulty': 'medium'}`


(50, 100)
Invoking: `DifficultyMap` with `{'player_levels': [1, 1], 'desired_difficulty': 'hard'}`


(100, 150)
Invoking: `DifficultyMap` with `{'player_levels': [1, 1], 'desired_difficulty': 'deadly'}`


(150, 200)For a level 1 fighter and a level 1 ranger adventuring in a busy marketplace, we will design an adventure with escalating drama and difficulty, ensuring each encounter builds off the previous one. The XP thresholds for encounters of varying difficulties are as follows: Easy (0-50 XP), Medium (50-100 XP), Hard (100-150 XP), and Deadly (150-200 XP).

### Encounter 1: The Pickpocket (Easy)
- **Story Hook**: As the party browses the bustling marketplace, a nimble pickpocket attempts to steal from them.
- **Enemies**: 1x Bandit (CR 1/8, 25 XP)
- **Tot

In [51]:
with open("output.txt", "w") as fh:
    fh.write(res["output"])